<a href="https://colab.research.google.com/github/Gauravsaha-97/Text_Generator_LSTM/blob/master/text_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/irish-lyrics-eof.txt \
    -O /tmp/irish-lyrics-eof.txt

--2020-07-23 09:16:47--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/irish-lyrics-eof.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 172.253.117.128, 173.194.202.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68970 (67K) [text/plain]
Saving to: ‘/tmp/irish-lyrics-eof.txt’

/tmp/irish-lyrics-e 100%[===================>]  67.35K  --.-KB/s    in 0.001s  

2020-07-23 09:16:47 (101 MB/s) - ‘/tmp/irish-lyrics-eof.txt’ saved [68970/68970]



In [4]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
data = open('/tmp/irish-lyrics-eof.txt').read()

In [13]:
tokenizer = Tokenizer()
corpus = data.lower().split("/n")
tokenizer.fit_on_texts(corpus)
word_index = tokenizer.word_index
total_words = len(tokenizer.word_index)+1

In [15]:
print("Total number of words is {}".format(total_words))

Total number of words is 2690


In [20]:
input_sequences = []
for text in corpus:
  text_sequence = tokenizer.texts_to_sequences([text])[0]
  for i in range(1,len(text_sequence)):
    n_seq = text_sequence[:i+1]
    input_sequences.append(n_seq)
print(input_sequences)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [22]:
input_sequences[1]

[51, 12, 96]

In [23]:
max_seq_length = max([len(x) for x in input_sequences])
print("Max sequence length is {}".format(max_seq_length))

Max sequence length is 13730


In [24]:
padding_seq = pad_sequences(input_sequences, padding='pre',maxlen=max_seq_length)
padding_seq

array([[   0,    0,    0, ...,    0,   51,   12],
       [   0,    0,    0, ...,   51,   12,   96],
       [   0,    0,    0, ...,   12,   96, 1217],
       ...,
       [   0,    0,   51, ...,  324,   47,  105],
       [   0,   51,   12, ...,   47,  105,  138],
       [  51,   12,   96, ...,  105,  138,  184]], dtype=int32)

In [33]:
xs, labels = padding_seq[:,:-1],padding_seq[:,-1]
labels = np.array(labels)
sen_labels = to_categorical(labels, num_classes=total_words)
sen_labels = np.array(sen_labels)

In [34]:
xs[6]

array([ 0,  0,  0, ..., 48,  2, 69], dtype=int32)

In [35]:
sen_labels[6]

array([0., 0., 1., ..., 0., 0., 0.], dtype=float32)

In [30]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Embedding(total_words, 100, input_length=max_seq_length),
                                    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
                                    tf.keras.layers.LSTM(40),
                                    tf.keras.layers.Dense(total_words*2, activation='relu'),
                                    tf.keras.layers.Dense(total_words, activation='softmax')

])

In [31]:
Adam = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=Adam, loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(xs, sen_labels, epochs=500)